Get contract ranges

In [2]:
# import kagglehub
import json
import math

# # Download latest version

#Run this code to get the path to the 2 JSON files, and add both of them to the Group project folder
# path = kagglehub.dataset_download("zynicide/nfl-football-player-stats")
# path2 = kagglehub.dataset_download("nicholasliusontag/nfl-contract-and-draft-data")


# print("Path to dataset files:", path)
# print("Path to dataset files:", path2)

file_path = "/Users/rshiknis/CS 4641/SalaryPrediction/cs4641-group-48" #ADD file path to "group project folder"

games_stats = open(f"{file_path}/games_1512362753.8735218.json")
games_json = json.load(games_stats)

profiles_stats = open(f"{file_path}/profiles_1512362725.022629.json")
profiles_json = json.load(profiles_stats)

contracts_data = open(f"{file_path}/combined_data_2000-2023.csv", "r")

min_year = dict()
contract_years = dict()

for i, line in enumerate(contracts_data):
    if i == 0:
        continue

    fields = line.split(',')
    contract_player_name = fields[4]
    contract_position = fields[5]
    contract_year = math.floor(float(fields[10]))

    # Only include contract years 2017 and earlier
    if contract_year <= 2017:
        min_year[contract_player_name] = min(contract_year, min_year.get(contract_player_name, float('inf')))
        if contract_player_name not in contract_years:
            contract_years[contract_player_name] = []
        if contract_year not in contract_years[contract_player_name]:
            contract_years[contract_player_name].append(contract_year)

year_ranges = dict()

for player in contract_years:
    year_ranges[player] = dict()

    contract_years[player].sort()

    for i in range(len(contract_years[player]) - 1):
        current_min_year = contract_years[player][i]
        current_range = [current_min_year]
        current_year = current_min_year + 1

        while current_year not in contract_years[player]:
            current_range.append(current_year)
            current_year += 1
        
        year_ranges[player][contract_years[player][i + 1]] = current_range

#print(year_ranges['Drew Brees'])


Calculate total stats prior to contract period for each player

In [ ]:
import pandas as pd
games_stats = open(f"{file_path}/games_1512362753.8735218.json")
games_json = json.load(games_stats)

profiles_stats = open(f"/{file_path}/profiles_1512362725.022629.json")
profiles_json = json.load(profiles_stats)

contracts_data = open(f"{file_path}/combined_data_2000-2023.csv", "r")

#f = open("output.csv", "a")
#f.write("contract_player_name,contract_year,contract_value,total_passing_attempts,total_passing_completions,total_passing_yards,total_passing_rating,total_passing_touchdowns,total_passing_interceptions,total_passing_sacks,total_passing_sacks_yards_lost,total_rushing_attempts,total_rushing_yards,total_rushing_touchdowns,total_games/n")
data = []
for i, line in enumerate(contracts_data):
    if i == 0:
        continue

    fields = line.split(',')
    contract_player_name = fields[4]
    contract_position = fields[5]
    contract_value = fields[12]
    contract_year = math.floor(float(fields[10]))

    if contract_year >= 2018:
        continue

    
    for profile in profiles_json:
        if contract_player_name == profile['name'] and contract_position == profile['position'] and profile['position'] == 'QB':
            total_passing_attempts = 0
            total_passing_completions = 0
            total_passing_yards = 0
            total_passing_rating = 0
            total_passing_touchdowns = 0
            total_passing_interceptions = 0
            total_passing_sacks = 0
            total_passing_sacks_yards_lost = 0
            total_rushing_attempts = 0
            total_rushing_yards = 0
            total_rushing_touchdowns = 0

            total_games = 0

            for game_player in games_json:
                if profile['player_id'] == game_player['player_id'] and (contract_year in year_ranges[profile['name']] and int(game_player['year']) in year_ranges[profile['name']][contract_year]):
                    passing_attempts = game_player['passing_attempts']
                    passing_completions = game_player['passing_completions']
                    passing_yards = game_player['passing_yards']
                    passing_rating = game_player['passing_rating']
                    passing_touchdowns = game_player['passing_touchdowns']
                    passing_interceptions = game_player['passing_interceptions']
                    passing_sacks = game_player['passing_sacks']
                    passing_sacks_yards_lost = game_player['passing_sacks_yards_lost']
                    rushing_attempts = game_player['rushing_attempts']
                    rushing_yards = game_player['rushing_yards']
                    rushing_touchdowns = game_player['rushing_touchdowns']

                    total_passing_attempts += passing_attempts
                    total_passing_completions += passing_completions
                    total_passing_yards += passing_yards
                    total_passing_rating += passing_rating
                    total_passing_touchdowns += passing_touchdowns
                    total_passing_interceptions += passing_interceptions
                    total_passing_sacks += passing_sacks
                    total_passing_sacks_yards_lost += passing_sacks_yards_lost
                    total_rushing_attempts += rushing_attempts
                    total_rushing_yards += rushing_yards
                    total_rushing_touchdowns += rushing_touchdowns
                    total_games += 1
            if total_games > 0:
                total_passing_rating /= total_games
                data_line = f"{contract_player_name},{contract_year},{contract_value},{total_passing_attempts},{total_passing_completions},{total_passing_yards},{total_passing_rating},{total_passing_touchdowns},{total_passing_interceptions},{total_passing_sacks},{total_passing_sacks_yards_lost},{total_rushing_attempts},{total_rushing_yards},{total_rushing_touchdowns},{total_games}/n"
                data.append({'contract_player_name': contract_player_name, 
                             'contract_year': contract_year,
                             'contract_value': contract_value,
                             'total_passing_attempts': total_passing_attempts,
                             'total_passing_completions': total_passing_completions,
                             'total_passing_yards': total_passing_yards,
                             'average_passing_rating': total_passing_rating, 
                             'total_passing_touchdowns': total_passing_touchdowns, 
                             'total_passing_interceptions': total_passing_interceptions, 
                             'total_passing_sacks': total_passing_sacks,
                             'total_passing_sacks_yards_lost': total_passing_sacks_yards_lost,
                             'total_rushing_attempts': total_rushing_attempts,
                             'total_rushing_yards': total_rushing_yards,
                             'total_rushing_touchdowns': total_rushing_touchdowns,
                             'total_games_played': total_games
                             })
                #print(data)
            
    modeling_df = pd.DataFrame(data)
    #data_line = f"{contract_player_name}, contract year: {contract_year}, contract value: {contract_value}, pass att: {total_passing_attempts}, pass comp: {total_passing_completions}, pass yards: {total_passing_yards}, pass rating: {total_passing_rating}, pass td: {total_passing_touchdowns}, ints: {total_passing_interceptions}, sacks: {total_passing_sacks}, sack yards lost: {total_passing_sacks_yards_lost}, rush att: {total_rushing_attempts}, rush yards: {total_rushing_yards}, rush td: {total_rushing_touchdowns}, games played: {total_games}/n"
    modeling_df.to_csv(f'{file_path}/initial_contractYrs_data.csv')        

#f.close()

Games for Specific Player

In [ ]:
import json

# Load data files
games_stats = open(f"{file_path}/games_1512362753.8735218.json")
games_json = json.load(games_stats)

profiles_stats = open(f"{file_path}/profiles_1512362725.022629.json")
profiles_json = json.load(profiles_stats)

# Find specific player ID
player_name_id = None
for profile in profiles_json:
    if profile['name'] == "Jimmy Garoppolo" and profile['position'] == 'QB':
        player_name_id = profile['player_id']
        break

if player_name_id:
    print("Jimmy Garoppolo' Game Stats:")

    for game in games_json:
        if game['player_id'] == player_name_id and int(game['year']) < 2018:
            # Retrieve stats for this game
            passing_attempts = game.get('passing_attempts', 0)
            passing_completions = game.get('passing_completions', 0)
            passing_yards = game.get('passing_yards', 0)
            passing_rating = game.get('passing_rating', 0)
            passing_touchdowns = game.get('passing_touchdowns', 0)
            passing_interceptions = game.get('passing_interceptions', 0)
            passing_sacks = game.get('passing_sacks', 0)
            passing_sacks_yards_lost = game.get('passing_sacks_yards_lost', 0)
            rushing_attempts = game.get('rushing_attempts', 0)
            rushing_yards = game.get('rushing_yards', 0)
            rushing_touchdowns = game.get('rushing_touchdowns', 0)

            # Print game stats
            print(f"Game Date: {game.get('date', 'Unknown Date')}")
            print(f"  Passing Attempts: {passing_attempts}")
            print(f"  Passing Completions: {passing_completions}")
            print(f"  Passing Yards: {passing_yards}")
            print(f"  Passing Rating: {passing_rating}")
            print(f"  Passing Touchdowns: {passing_touchdowns}")
            print(f"  Interceptions: {passing_interceptions}")
            print(f"  Sacks: {passing_sacks}")
            print(f"  Sack Yards Lost: {passing_sacks_yards_lost}")
            print(f"  Rushing Attempts: {rushing_attempts}")
            print(f"  Rushing Yards: {rushing_yards}")
            print(f"  Rushing Touchdowns: {rushing_touchdowns}")
            print("\n")
else:
    print("player profile not found in the data.")




Jimmy Garoppolo' Game Stats:
Game Date: 2014-09-29
  Passing Attempts: 6
  Passing Completions: 7
  Passing Yards: 70
  Passing Rating: 147.9
  Passing Touchdowns: 1
  Interceptions: 0
  Sacks: 1
  Sack Yards Lost: 6
  Rushing Attempts: 0
  Rushing Yards: 0
  Rushing Touchdowns: 0


Game Date: 2014-10-05
  Passing Attempts: 0
  Passing Completions: 0
  Passing Yards: 0
  Passing Rating: 0
  Passing Touchdowns: 0
  Interceptions: 0
  Sacks: 0
  Sack Yards Lost: 0
  Rushing Attempts: 3
  Rushing Yards: -4
  Rushing Touchdowns: 0


Game Date: 2014-10-26
  Passing Attempts: 3
  Passing Completions: 3
  Passing Yards: 22
  Passing Rating: 97.2
  Passing Touchdowns: 0
  Interceptions: 0
  Sacks: 1
  Sack Yards Lost: 11
  Rushing Attempts: 1
  Rushing Yards: -1
  Rushing Touchdowns: 0


Game Date: 2014-11-16
  Passing Attempts: 0
  Passing Completions: 0
  Passing Yards: 0
  Passing Rating: 0
  Passing Touchdowns: 0
  Interceptions: 0
  Sacks: 0
  Sack Yards Lost: 0
  Rushing Attempts: 2
  Ru